In [2]:
from src.model.eegnet_variance import EEGNetMultiHeaded
from pytorch_lightning.loggers import CometLogger
from src.datamodule import DataModule
from src.preprocessing import create_dataset
import pytorch_lightning as pl
from src.preprocessing import *
from src.noise_addition import zero_signal, add_gaussian_noise
import copy
import random
import pickle
from src.model.eegnet_variance import EEGNetMultiHeaded
from pytorch_lightning.loggers import CometLogger
from src.datamodule import DataModule
from src.preprocessing import create_dataset
import pytorch_lightning as pl
from src.preprocessing import *
from src.noise_addition import zero_signal, add_gaussian_noise
from src.model.variance_wrapper import EEGVarianceWrapper
import copy
import random
import pickle
import torch
import shap
import os.path

c:\Users\marti_wfv2bf9\AppData\Local\Programs\Python\Python39\lib\site-packages\shap\utils\_clustering.py:35: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-jit for details.
  def _pt_shuffle_rec(i, indexes, index_mask, partition_tree, M, pos):
c:\Users\marti_wfv2bf9\AppData\Local\Programs\Python\Python39\lib\site-packages\shap\utils\_clustering.py:54: NumbaDeprecationWarning: The 'nopython' keyword argument was not supplied to the 'numba.jit' decorator. The implicit default value for this argument is currently False, but it will be changed to True in Numba 0.59.0. See https://numba.readthedocs.io/en/stable/reference/deprecation.html#deprecation-of-object-mode-fall-back-behaviour-when-using-ji

In [3]:
comet_logger = CometLogger(
    api_key="WSATCNWE43zphHslQCTsJKcgk",
    workspace="marwo22",  # Optional
    project_name="bachelors-project"  # Optional
)

CometLogger will be initialized in online mode


In [4]:
def add_guassian_noise_to_dataset(dataset, severity_index, low: int = 0, high: int = 308):
    if (severity_index == 0):
        return dataset

    length = len(dataset[0])
        # Add noise to 7.5%  * severityIndex of samples in the dataset. It can thus range from 5% to 50%
    episodes_to_corrupt = random.sample(range(0, length), int(0.1 * severity_index * length))
        # Add noise to the samples
    for episode in episodes_to_corrupt:
        channels_to_corrupt = int(64 / 10 * severity_index)
        dataset[0][episode] = add_gaussian_noise(dataset[0][episode], 2 * severity_index, channels_to_corrupt, low, high)

    return dataset

In [5]:
def add_zero_to_dataset(dataset, severity_index, low: int = 0, high: int = 308):
    if (severity_index == 0):
        return dataset
    
    length = len(dataset[0])
        # Add noise to 7.5%  * severityIndex of samples in the dataset. It can thus range from 5% to 50%
    episodes_to_corrupt = random.sample(range(0, length), int(0.1 * severity_index * length))
        # Add noise to the samples
    for episode in episodes_to_corrupt:
            # Can range from 5-50%
        channels_to_corrupt = int(64 / 20 * severity_index)
            # Zeroes the entire signal
        dataset[0][episode] = zero_signal(dataset[0][episode], channels_to_corrupt, low, high, 100)

    return dataset

In [ ]:
# Tracable noise experiment:

if os.path.isfile('results_pickled/checkpoints/shap.pkl'):
    with open('results_pickled/checkpoints/shap.pkl', 'rb') as f:
        progress = pickle.load(f)
    with open('results_pickled/shap_values_zeroed.pkl', 'rb') as f:
        shap_values_zeroed = pickle.load(f)

    with open('results_pickled/shap_values_gaussian.pkl', 'rb') as f:
        shap_values_gaussian = pickle.load(f)
    print(progress)
    
else:
    shap_values_gaussian = [[] for _ in range(3)]
    shap_values_zeroed = [[] for _ in range(3)]
    progress = 1

_, _, test = create_dataset('./src/pickle_df', 1)


positive_indices = np.where(np.asarray(test[1]) == 1)[0]
test_indices = np.random.choice(positive_indices, size=10, replace=False)
print(test_indices)

while progress < 3:  
    # Load in the datasets with the current participant as test set
    train, val, _ = create_dataset('./src/pickle_df', 1)

    dm = DataModule(train=train, val=val, test=test, batch_size=16)

    test_shap = list()

    shap_test_tensor = torch.empty((0, 1, 64, 308), dtype=torch.float32)

    for (data, _) in dm.test_dataloader():
        shap_test_tensor = torch.cat((shap_test_tensor, data), 0)

    test_episodes = shap_test_tensor[test_indices]
    background = np.delete(shap_test_tensor, test_indices, axis=0)

    model = EEGNetMultiHeaded(chunk_size=308,
                            num_electrodes=64,
                            dropout=0.5,
                            kernel_1=64,
                            kernel_2=16,
                            F1=8,
                            F2=16,
                            D=2,
                            num_classes=2)
    # Train for 25 epochs for this example
    # Final one for results wil run for 50 most likely
    trainer = pl.Trainer(
        max_epochs=25,
        logger=comet_logger
    )

    # Fit and test model
    trainer.fit(model, datamodule=dm)

    variance_model = EEGVarianceWrapper(model)

    e_variance = shap.DeepExplainer(variance_model, background)

    shap_values_variance = e_variance.shap_values(test_episodes)

    shap_values_gaussian[0].append(shap_values_variance[1])
    shap_values_zeroed[0].append(shap_values_variance[1])


    for i in range(2):
        # Take deep copies
        train_copy = copy.deepcopy(train)
        val_copy = copy.deepcopy(val)
        test_copy = copy.deepcopy(test)
        
        # Add noise to the deep copies. If i is 0, noise is added to the lower end, when it is 1, it is added to the higher end
        # For this experiment, the intensity value is 10, resulting in a 'high' level of noise

        train_copy = add_guassian_noise_to_dataset(train_copy, 10, low = (i) * 128, high = (i + 1) * 128 - 1)
        val_copy = add_guassian_noise_to_dataset(val_copy, 10, low = (i) * 128, high = (i + 1) * 128 - 1)
        test_copy = add_guassian_noise_to_dataset(test_copy, 10, low = (i) * 128, high = (i + 1) * 128 - 1)

        dm = DataModule(train=train_copy, val=val_copy, test=test_copy, batch_size=16)

        model = EEGNetMultiHeaded(chunk_size=308,
                                num_electrodes=64,
                                dropout=0.5,
                                kernel_1=64,
                                kernel_2=16,
                                F1=8,
                                F2=16,
                                D=2,
                                num_classes=2)
        # Train for 25 epochs for this example
        # Final one for results wil run for 50 most likely
        trainer = pl.Trainer(
            max_epochs=25,
            logger=comet_logger
        )

        # Fit and test model
        trainer.fit(model, datamodule=dm)

        variance_model = EEGVarianceWrapper(model)

        test_shap = list()

        shap_test_tensor = torch.empty((0, 1, 64, 308), dtype=torch.float32)

        for (data, _) in dm.test_dataloader():
            shap_test_tensor = torch.cat((shap_test_tensor, data), 0)

        test_episodes = shap_test_tensor[test_indices]
        background = np.delete(shap_test_tensor, test_indices, axis=0)


        e_variance = shap.DeepExplainer(variance_model, background)

        shap_values_variance = e_variance.shap_values(test_episodes)

        shap_values_gaussian[i + 1].append(shap_values_variance[1])

        # Take deep copies
        train_copy = copy.deepcopy(train)
        val_copy = copy.deepcopy(val)
        test_copy = copy.deepcopy(test)
        
        # Add noise to the deep copies. If i is 0, noise is added to the lower end, when it is 1, it is added to the higher end
        # For this experiment, the intensity value is 5, resulting in a 'medium' level of noise

        train_copy = add_zero_to_dataset(train_copy, 10, low = (i - 1) * 128, high = i * 128 - 1)
        val_copy = add_zero_to_dataset(val_copy, 10, low = (i - 1) * 128, high = i * 128 - 1)
        test_copy = add_zero_to_dataset(test_copy, 10, low = (i - 1) * 128, high = i * 128 - 1)

        dm = DataModule(train=train_copy, val=val_copy, test=test_copy, batch_size=16)

        model = EEGNetMultiHeaded(chunk_size=308,
                                num_electrodes=64,
                                dropout=0.5,
                                kernel_1=64,
                                kernel_2=16,
                                F1=8,
                                F2=16,
                                D=2,
                                num_classes=2)
        # Train for 25 epochs for this example
        # Final one for results wil run for 50 most likely
        trainer = pl.Trainer(
            max_epochs=25,
            logger=comet_logger
        )

        # Fit and test model
        trainer.fit(model, datamodule=dm)

        variance_model = EEGVarianceWrapper(model)

        test_shap = list()

        shap_test_tensor = torch.empty((0, 1, 64, 308), dtype=torch.float32)

        for (data, _) in dm.test_dataloader():
            shap_test_tensor = torch.cat((shap_test_tensor, data), 0)

        test_episodes = shap_test_tensor[test_indices]
        background = np.delete(shap_test_tensor, test_indices, axis=0)


        e_variance = shap.DeepExplainer(variance_model, background)

        shap_values_variance = e_variance.shap_values(test_episodes)

        shap_values_zeroed[i + 1].append(shap_values_variance[1])

    progress += 1

    # Save them to a pkl file
    with open('results_pickled/shap_values_gaussian.pkl', 'wb') as f:
        pickle.dump(shap_values_gaussian, f)  

    # Save them to a pkl file
    with open('results_pickled/shap_values_zeroed.pkl', 'wb') as f:
        pickle.dump(shap_values_zeroed, f)
    
    with open('results_pickled/results_sets/tracable_' + str(progress) + '.pkl', 'wb') as f:
        pickle.dump([train, val, test], f)

    with open('results_pickled/checkpoints/shap.pkl', 'wb') as f:
        pickle.dump(progress, f)

    # Just to visualize progress easier, the output is very messy
    print("\n\n\n\n\nFinished" + str(progress) + "\n\n\n\n\n")

del test

In [ ]:
# General noise experiment:

if os.path.isfile('results_pickled/checkpoints/global.pkl'):
    with open('results_pickled/checkpoints/global.pkl', 'rb') as f:
        progress = pickle.load(f)

    with open('results_pickled/global_variances_gaussian.pkl', 'rb') as f:
        variances_guassian = pickle.load(f)

    with open('results_pickled/global_accuracies_gaussian.pkl', 'rb') as f:
        accuracies_gaussian = pickle.load(f)

    with open('results_pickled/global_variances_zeroed.pkl', 'rb') as f:
        variances_zeroed = pickle.load(f)

    with open('results_pickled/global_accuracies_zeroed.pkl', 'rb') as f:
        accuracies_zeroed = pickle.load(f)

    print(progress)

else:
    progress = 0

    variances_guassian = [[] for _ in range(11)]
    accuracies_gaussian = [[] for _ in range(11)]

    variances_zeroed = [[] for _ in range(11)]
    accuracies_zeroed = [[] for _ in range(11)]
while progress < 6:
    # Load in the datasets with the current participant as test set
    train, val, test = create_dataset('./src/pickle_df', 1)

    for i in range(11):
        # Take deep copies
        train_copy = copy.deepcopy(train)
        val_copy = copy.deepcopy(val)
        test_copy = copy.deepcopy(test)
        
        # Add noise to the deep copies. If i is 0, noise is added to the lower end, when it is 1, it is added to the higher end
        # For this experiment, the intensity value is 5, resulting in a 'medium' level of noise
        train_copy = add_guassian_noise_to_dataset(train_copy, i)
        val_copy = add_guassian_noise_to_dataset(val_copy, i)
        test_copy = add_guassian_noise_to_dataset(test_copy, i)

        dm = DataModule(train=train_copy, val=val_copy, test=test_copy, batch_size=16)

        model = EEGNetMultiHeaded(chunk_size=308,
                                num_electrodes=64,
                                dropout=0.5,
                                kernel_1=64,
                                kernel_2=16,
                                F1=8,
                                F2=16,
                                D=2,
                                num_classes=2)
        # Train for 25 epochs for this example
        # Final one for results wil run for 50 most likely
        trainer = pl.Trainer(
            max_epochs=25,
            logger=comet_logger
        )

        # Fit and test model
        trainer.fit(model, datamodule=dm)
        test_results = trainer.test(model, datamodule=dm)
        
        # Capture the testing variance and accuracy
        variances_guassian[i].append(test_results[0]['overall_var'])
        accuracies_gaussian[i].append(test_results[0]['test_acc'])

        # Take deep copies
        train_copy = copy.deepcopy(train)
        val_copy = copy.deepcopy(val)
        test_copy = copy.deepcopy(test)
        
        # Add noise to the deep copies. If i is 0, noise is added to the lower end, when it is 1, it is added to the higher end
        # For this experiment, the intensity value is 5, resulting in a 'medium' level of noise
        train_copy = add_zero_to_dataset(train, i)
        val_copy = add_zero_to_dataset(val_copy, i)
        test_copy = add_zero_to_dataset(test_copy, i)

        dm = DataModule(train=train_copy, val=val_copy, test=test_copy, batch_size=16)

        model = EEGNetMultiHeaded(chunk_size=308,
                                num_electrodes=64,
                                dropout=0.5,
                                kernel_1=64,
                                kernel_2=16,
                                F1=8,
                                F2=16,
                                D=2,
                                num_classes=2)
        # Train for 25 epochs for this example
        # Final one for results wil run for 50 most likely
        trainer = pl.Trainer(
            max_epochs=25,
            logger=comet_logger
        )

        # Fit and test model
        trainer.fit(model, datamodule=dm)
        test_results = trainer.test(model, datamodule=dm)
        
        # Capture the testing variance and accuracy
        variances_zeroed[i].append(test_results[0]['overall_var'])
        accuracies_zeroed[i].append(test_results[0]['test_acc'])

    progress += 1

    # Save them to a pkl file
    with open('results_pickled/global_variances_gaussian.pkl', 'wb') as f:
        pickle.dump(variances_guassian, f)

    with open('results_pickled/global_accuracies_gaussian.pkl', 'wb') as f:
        pickle.dump(accuracies_gaussian, f)

    # Save them to a pkl file
    with open('results_pickled/global_variances_zeroed.pkl', 'wb') as f:
        pickle.dump(variances_zeroed, f)

    with open('results_pickled/global_accuracies_zeroed.pkl', 'wb') as f:
        pickle.dump(accuracies_zeroed, f)

    with open('results_pickled/checkpoints/global.pkl', 'wb') as f:
        pickle.dump(progress, f)
    
    with open('results_pickled/results_sets/global_' + str(progress) + '.pkl', 'wb') as f:
        pickle.dump([train, val, test], f)

    # Just to visualize progress easier, the output is very messy
    print("\n\n\n\n\nFinished" + str(progress) + "\n\n\n\n\n")

In [6]:
# Localized noise experiment:

if os.path.isfile('results_pickled/checkpoints/local.pkl'):
    with open('results_pickled/checkpoints/local.pkl', 'rb') as f:
        progress = pickle.load(f)

    with open('results_pickled/local_variances_gaussian.pkl', 'rb') as f:
        variances_guassian = pickle.load(f)

    with open('results_pickled/local_accuracies_gaussian.pkl', 'rb') as f:
        accuracies_gaussian = pickle.load(f)

    with open('results_pickled/local_variances_zeroed.pkl', 'rb') as f:
        variances_zeroed = pickle.load(f)

    with open('results_pickled/local_accuracies_zeroed.pkl', 'rb') as f:
        accuracies_zeroed = pickle.load(f)

    print(progress)

else:
    progress = 0

    variances_gaussian = [[] for _ in range(2)]
    accuracies_guassian = [[] for _ in range(2)]

    variances_zeroed = [[] for _ in range(2)]
    accuracies_zeroed = [[] for _ in range(2)]

while progress < 6:
    # Load in the datasets with the current participant as test set
    train, val, test = create_dataset('./src/pickle_df', 1)

    for i in range(2):
        # Take deep copies
        train_copy = copy.deepcopy(train)
        val_copy = copy.deepcopy(val)
        test_copy = copy.deepcopy(test)
        
        # Add noise to the deep copies. If i is 0, noise is added to the lower end, when it is 1, it is added to the higher end
        # For this experiment, the intensity value is 5, resulting in a 'medium' level of noise
        train_copy = add_guassian_noise_to_dataset(train_copy, 10, low = i * 102, high = (i + 1) * 102 - 1)
        val_copy = add_guassian_noise_to_dataset(val_copy, 10, low = i * 102, high = (i + 1) * 102 - 1)
        test_copy = add_guassian_noise_to_dataset(test_copy, 10, low = i * 102, high = (i + 1) * 102 - 1)

        dm = DataModule(train=train_copy, val=val_copy, test=test_copy, batch_size=16)

        model = EEGNetMultiHeaded(chunk_size=308,
                                num_electrodes=64,
                                dropout=0.5,
                                kernel_1=64,
                                kernel_2=16,
                                F1=8,
                                F2=16,
                                D=2,
                                num_classes=2)
        # Train for 25 epochs for this example
        # Final one for results wil run for 50 most likely
        trainer = pl.Trainer(
            max_epochs=25,
            logger=comet_logger
        )

        # Fit and test model
        trainer.fit(model, datamodule=dm)
        test_results = trainer.test(model, datamodule=dm)
        
        # Capture the testing variance and accuracy
        variances_gaussian[i].append(test_results[0]['overall_var'])
        accuracies_guassian[i].append(test_results[0]['test_acc'])

        # Take deep copies
        train_copy = copy.deepcopy(train)
        val_copy = copy.deepcopy(val)
        test_copy = copy.deepcopy(test)
        
        # Add noise to the deep copies. If i is 0, noise is added to the lower end, when it is 1, it is added to the higher end
        # For this experiment, the intensity value is 5, resulting in a 'medium' level of noise
        train_copy = add_zero_to_dataset(train_copy, 10, low = i * 102, high = (i + 1) * 102 - 1)
        val_copy = add_zero_to_dataset(val_copy, 10, low = i * 102, high = (i + 1) * 102 - 1)
        test_copy = add_zero_to_dataset(test_copy, 10, low = i * 102, high = (i + 1) * 102 - 1)

        dm = DataModule(train=train_copy, val=val_copy, test=test_copy, batch_size=16)

        model = EEGNetMultiHeaded(chunk_size=308,
                                num_electrodes=64,
                                dropout=0.5,
                                kernel_1=64,
                                kernel_2=16,
                                F1=8,
                                F2=16,
                                D=2,
                                num_classes=2)
        # Train for 25 epochs for this example
        # Final one for results wil run for 50 most likely
        trainer = pl.Trainer(
            max_epochs=25,
            logger=comet_logger
        )

        # Fit and test model
        trainer.fit(model, datamodule=dm)
        test_results = trainer.test(model, datamodule=dm)
        
        # Capture the testing variance and accuracy
        variances_zeroed[i].append(test_results[0]['overall_var'])
        accuracies_zeroed[i].append(test_results[0]['test_acc'])
        
    progress += 1

     # Save them to a pkl file
    with open('results_pickled/localized_variances_gaussian.pkl', 'wb') as f:
        pickle.dump(variances_gaussian, f)

    with open('results_pickled/localized_accuracies_gaussian.pkl', 'wb') as f:
        pickle.dump(accuracies_guassian, f)

    # Save them to a pkl file
    with open('results_pickled/localized_variances_zeroed.pkl', 'wb') as f:
        pickle.dump(variances_zeroed, f)

    with open('results_pickled/localized_accuracies_zeroed.pkl', 'wb') as f:
        pickle.dump(accuracies_zeroed, f)

    with open('results_pickled/checkpoints/local.pkl', 'wb') as f:
        pickle.dump(progress, f)
    
    with open('results_pickled/results_sets/local_' + str(progress) + '.pkl', 'wb') as f:
        pickle.dump([train, val, test], f)

    # Just to visualize progress easier, the output is very messy
    print("\n\n\n\n\nFinished" + str(progress) + "\n\n\n\n\n")


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name             | Type            | Params
-----------------------------------------------------
0 | block1           | Sequential      | 1.6 K 
1 | block2           | Sequential      | 544   
2 | mean_node        | Linear          | 288   
3 | variance_node    | Sequential      | 288   
4 | sampling_softmax | SamplingSoftmax | 0     
-----------------------------------------------------
2.7 K     Trainable params
0         Non-trainable params
2.7 K     Total params
0.011     Total estimated model params size (MB)


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/marwo22/bachelors-project/af42a02288e64ae48ca723215a1383c0



The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 6 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.


Epoch 24: 100%|██████████| 431/431 [00:04<00:00, 88.41it/s, v_num=83c0] 

...

COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experiment is live on comet.com https://www.comet.com/marwo22/bachelors-project/af42a02288e64ae48ca723215a1383c0



Testing DataLoader 0: 100%|██████████| 66/66 [00:02<00:00, 30.61it/s]


┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│        correct_var        │    0.24772694750194682    │
│       incorrect_var       │    0.24863874685504778    │
│        overall_var        │    0.24809414148330688    │
│         test_acc          │    0.6513410210609436     │
│         test_loss         │    15.843411445617676     │
└───────────────────────────┴───────────────────────────┘

COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO: Comet.ml ExistingExperiment Summary
COMET INFO: ---------------------------------------------------------------------------------------
COMET INFO:   Data:
COMET INFO:     display_summary_level : 1
COMET INFO:     url                   : https://www.comet.com/marwo22/bachelors-project/af42a02288e64ae48ca723215a1383c0
COMET INFO:   Metrics:
COMET INFO:     correct_var   : 0.24772694750194682
COMET INFO:     incorrect_var : 0.24863874685504778
COMET INFO:     overall_var   : 0.24809414148330688
COMET INFO:     test_acc      : 0.6513410210609436
COMET INFO:     test_loss     : 15.843411445617676
COMET INFO:   Others:
COMET INFO:     Created from : pytorch-lightning
COMET INFO: 
COMET WARNING: To get all data logged automatically, import comet_ml before the following modules: torch.
COMET INFO: Uploading 22 metrics, params and output messages







Finished6





